In [1146]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd

In [1147]:
#reading rheast and extracting districts
tamsat= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
preharvest=pd.read_csv('data/preharvest/preharvest_cleaned.csv')
postharvest=pd.read_csv('data/postharvest/postharvest_cleaned.csv')


In [1148]:
# # remove all white spaces, turn to lower case and
tamsat.cname=tamsat.cname.str.lower()
tamsat.cname = tamsat.cname.str.strip()
tamsat.cname=tamsat.cname.str.replace('-', '', regex=True)
tamsat.cname=tamsat.cname.str.replace(' ', '', regex=True)

In [1149]:
#changing date columns
tamsat['planting']=pd.to_datetime(tamsat['planting'], format='ISO8601')
tamsat['harvest'] = pd.to_datetime(tamsat['harvest'], format='ISO8601')
tamsat['year'] = tamsat['planting'].dt.year
tamsat['district']=tamsat['cname']
tamsat['yield']=tamsat['gwad']/1000


In [1150]:
#Derive seasons and add to rheas data
seasons = [1, 1, 1, 1,1, 0, 0, 0, 0, 1, 1, 1]
month_to_season=dict(zip(range(1,13), seasons))
tamsat['season1']= tamsat['planting'].dt.month.map(month_to_season)

# Extract data only for the planting season
tamsat_season= tamsat.loc[tamsat['season1']==1]

In [1151]:
# aggregate data
tamsat_yearly=tamsat_season.groupby(['district', 'season1', 'year'], as_index=False).agg({'yield':'mean'})
preharvest_yearly=preharvest.groupby(['district', 'year'], as_index=False).agg({'yield':'mean'})

In [1152]:
#Extracting Districts list
#first set cname to district
tamsat['district']=tamsat['cname']

preharvest_list_districts= preharvest['district'].to_list()
preharvest_set_districts=set(preharvest_list_districts)
preharvest_districts=list(preharvest_set_districts)
rheas_tamsat_list_districts=tamsat['district'].to_list()
rheas_tamsat_set_districts=set(rheas_tamsat_list_districts)
rheas_districts=list(rheas_tamsat_set_districts)

In [1153]:
#Finding common districts in rheas and preharvestharvest tamsat
districts=[]
for i in preharvest_districts:
    if i in rheas_districts:
        districts.append(i)
    else:
        pass

missing_districts=[]
for j in rheas_districts:
    if j not in  preharvest_districts:
        missing_districts.append(j)
    else:
        pass
len(missing_districts)

42

In [1154]:
# tamsat and preharvest metrics
# preharvest is 2011 to 2022
rheas_preharvest_districts=districts
district_rheas_prehavest=tamsat_yearly.loc[(tamsat_yearly['year']>=2011)&(tamsat_yearly['year']<=2022)]
rheas_preharvest_disct_check=[]
distric_metrics1=[]

for i in rheas_preharvest_districts:
        district_preharvest=preharvest_yearly.loc[preharvest_yearly['district']==i].reset_index(drop=True)
        rheas_tamsat_data3=district_rheas_prehavest.loc[district_rheas_prehavest['district']==i].reset_index(drop=True)
        #  pearson's correlation
        corr=rheas_tamsat_data3['yield'].corr(district_preharvest['yield'])
        # # #root mean square error
        rmse = np.sqrt(((rheas_tamsat_data3['yield']-district_preharvest['yield'])**2).mean())
        # # #unbiased RMSE
        ubrmse = np.sqrt((((rheas_tamsat_data3['yield']-rheas_tamsat_data3['yield'].mean())
                    -(district_preharvest['yield']-district_preharvest['yield'].mean()))**2).mean())
        # #bias
        bias = (rheas_tamsat_data3['yield']-district_preharvest['yield']).mean()
        distric_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
        # convert into dataframe and rename column
        rheas_preharvest_disct_check.append(district_preharvest)

check_rheas_preharvest_disct= pd.concat(rheas_preharvest_disct_check, ignore_index=True)
district_rheas_preharvest_metrics1=pd.DataFrame(distric_metrics1)
district_rheas_preharvest_metrics=district_rheas_preharvest_metrics1.rename({0:'district',1: 'correlation', 2: 
                                                    'rmse', 3:'ubrmse', 4:'bias'}, axis=1)

district_rheas_preharvest_metrics.to_csv('data/preharvestharvest_rheas__district_metrics.csv')

district_rheas_preharvest_metrics
check_rheas_preharvest_disct


,district,year,yield
0,nchelenge,2011,2.037484
1,nchelenge,2012,2.892034
2,nchelenge,2013,2.381748
3,nchelenge,2014,2.732410
4,nchelenge,2015,3.649793
...,...,...,...
881,mpongwe,2018,3.070317
882,mpongwe,2019,3.567245
883,mpongwe,2020,3.005421
884,mpongwe,2021,2.867502


In [1155]:
# Extract rheas data only for  districts in preharvest
districts
datasets=district_rheas_prehavest
district_rheas_pre_list=[]
for i in districts:
    rheast_pre_dist_df=district_rheas_prehavest.loc[district_rheas_prehavest['district']==i]
    district_rheas_pre_list.append(rheast_pre_dist_df)
rheas_pre_common=pd.concat(district_rheas_pre_list, ignore_index=True)
rheas_pre_common
# district_rheas_prehavest.loc[datasets['district']!='kalomo']

,district,season1,year,yield
0,nchelenge,1,2011,4.015630
1,nchelenge,1,2012,3.998510
2,nchelenge,1,2013,4.122770
3,nchelenge,1,2014,4.007900
4,nchelenge,1,2015,3.921880
...,...,...,...,...
883,mpongwe,1,2018,3.825260
884,mpongwe,1,2019,3.799420
885,mpongwe,1,2020,3.957850
886,mpongwe,1,2021,4.160970


In [1156]:
# rheas and preharvestharvest merged with metrics
# bias correcting tamsat using preharvest over the province
rheas_pre_common=rheas_pre_common.groupby(['district'], as_index=False).agg({'yield':'mean'})
preharvest_yearly=preharvest_yearly.groupby(['district'],as_index=False).agg({'yield':'mean'})

# marge preharvest, metrics and tamsat rheas yields
preharvest_yearly.rename(columns={'yield':'pre_yield'}, inplace=True)
district_rheas_preharvest_merged=pd.concat([rheas_pre_common,preharvest_yearly,district_rheas_preharvest_metrics], axis=1)

# calculate bias corrected yield
district_rheas_preharvest_merged['corrected_yield']=district_rheas_preharvest_merged['yield']-district_rheas_preharvest_merged['bias']
district_rheas_preharvest_merged

,district,yield,district,pre_yield,district,correlation,rmse,ubrmse,bias,corrected_yield
0,chadiza,3.969677,chadiza,2.510959,nchelenge,-0.174538,1.420562,0.441810,1.350111,2.619566
1,chama,3.987791,chama,2.639932,kaputa,-0.500639,1.727325,0.601713,1.619134,2.368657
2,chavuma,3.994793,chavuma,2.662906,milengi,0.388910,0.988598,0.244435,0.957902,3.036891
3,chibombo,3.971719,chibombo,2.541407,lufwanyama,0.201982,1.527334,0.378243,1.479757,2.491963
4,chiengi,3.969542,chiengi,2.605939,kazungula,0.407879,2.198775,0.244753,2.185110,1.784432
...,...,...,...,...,...,...,...,...,...,...
69,shangombo,3.977197,shangombo,0.992796,livingstone,0.409827,2.642838,0.763381,2.530187,1.447010
70,siavonga,3.984733,siavonga,1.806810,chadiza,0.353475,1.496571,0.334462,1.458718,2.526015
71,sinazongwe,3.981998,sinazongwe,1.665775,mwinilunga,-0.320575,0.559056,0.415781,0.373725,3.608273
72,solwezi,3.987743,solwezi,2.094350,isoka,0.312475,0.647218,0.278074,0.584437,3.403306


In [1126]:
# Province Level
# rename columns
preharvest['district_b']=preharvest['district']
tamsat_season['district_b']=tamsat_season['district']
postharvest['province_b']=postharvest['province']


#Read shapefiles and rename columns
zambia_district_shp = gpd.read_file('data/shp/zmb_district_cleaned.shp')
zambia_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_0.shp')
zambia_province_shp = gpd.read_file('data/shp/zmb_province_cleaned.shp')

# merge the yields and shapefiles by provinces
preharvest_shp= pd.merge(zambia_district_shp, preharvest, on="district_b")
post_shp= pd.merge(zambia_province_shp, postharvest, on="province_b")
rheas_tamsat_shp= pd.merge(zambia_district_shp, tamsat_season, on="district_b")

# aggregate preharvestharvest to provincial level 
preharvest_provincial=preharvest_shp.groupby(['province_b','province', 'year'], as_index=False).agg({'harvested_ha':'sum', 'production_mt': 
                                                             'sum','planted_ha':'sum' })
preharvest_provincial['yield']=preharvest_provincial['production_mt']/preharvest_provincial['harvested_ha']
preharvest_provincial['yield_planted']=preharvest_provincial['production_mt']/preharvest_provincial['planted_ha']
rheas_tamsat_province=rheas_tamsat_shp.groupby(['province_b', 'province', 'year'], as_index=False).agg({'yield':'mean'})
post_province=post_shp.groupby(['province_b', 'year'], as_index=False).agg({'yield':'mean'})

In [1127]:
provinces=['central', 'copperbelt', 'eastern', 'luapula', 'lusaka',
       'muchinga', 'northern', 'northwestern', 'southern', 'western']
rheas_provincial1=rheas_tamsat_province.loc[(rheas_tamsat_province['year']>=2011)&(rheas_tamsat_province['year']<=2022)]
rheas_preharvestharvest_province_metrics1=[]
rheas_preharvest_prov_check=[]
for i in provinces:
    preharvest_province1=preharvest_provincial.loc[preharvest_provincial['province_b']==i].reset_index(drop=True)
    rheas_tamsat_province1=rheas_provincial1.loc[rheas_provincial1['province_b']==i].reset_index(drop=True)  
    #  pearson's correlation
    corr=rheas_tamsat_province1['yield'].corr(preharvest_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheas_tamsat_province1['yield']-preharvest_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheas_tamsat_province1['yield']-rheas_tamsat_province1['yield'].mean())
                       -(preharvest_province1['yield']-preharvest_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheas_tamsat_province1['yield']-preharvest_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    rheas_preharvestharvest_province_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])
    rheas_preharvest_prov_check.append(preharvest_province1)

check_rheas_preharvest_prov= pd.concat(rheas_preharvest_prov_check, ignore_index=True)
rheas_preharvest_prov_metric=pd.DataFrame(rheas_preharvestharvest_province_metrics1)
rheas_preharvestharvest_province_metric=rheas_preharvest_prov_metric.rename({0:'province',1: 'correlation',2: 
                                                  'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
check_rheas_preharvest_prov

,province_b,province,year,harvested_ha,production_mt,planted_ha,yield,yield_planted
0,central,Central,2011,169991,558493,211185,3.285427,2.644568
1,central,Central,2012,156385,494215,184049,3.160246,2.685236
2,central,Central,2013,159372,478734,217001,3.003878,2.206137
3,central,Central,2014,217439,723762,256406,3.328575,2.822719
4,central,Central,2015,164809,484723,273008,2.941120,1.775490
...,...,...,...,...,...,...,...,...
115,western,Western,2018,85908,85107,118136,0.990676,0.720415
116,western,Western,2019,22961,22758,132120,0.991159,0.172252
117,western,Western,2020,90384,110203,130010,1.219276,0.847650
118,western,Western,2021,96459,110314,129580,1.143636,0.851320


In [1128]:
# rheas and preharvestharvest merged with metrics
# bias correcting tamsat using preharvest over the province
rheas_provincial1=rheas_provincial1.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province=post_province.groupby(['province_b'],as_index=False).agg({'yield':'mean'})

# marge preharvest, metrics and tamsat rheas yields
post_province.rename(columns={'yield':'post_yield'}, inplace=True)
province_rheas_preharvest_merged=pd.concat([rheas_provincial1,post_province,rheas_preharvestharvest_province_metric], axis=1)

# calculate bias corrected yield
province_rheas_preharvest_merged['corrected_yield']=province_rheas_preharvest_merged['yield']-province_rheas_preharvest_merged['bias']
province_rheas_preharvest_merged

,province_b,yield,province_b,post_yield,province,correlation,rmse,ubrmse,bias,corrected_yield
0,central,3.981736,central,2.078341,central,0.342367,1.200101,0.308748,1.159706,2.822031
1,copperbelt,3.987777,copperbelt,2.210160,copperbelt,0.033021,1.084943,0.185268,1.069007,2.918770
2,eastern,3.986621,eastern,1.768208,eastern,0.512175,1.785514,0.156952,1.778602,2.208019
3,luapula,3.987051,luapula,2.554374,luapula,0.249760,1.000830,0.173999,0.985588,3.001463
4,lusaka,3.982445,lusaka,2.115439,lusaka,0.215878,1.214629,0.347690,1.163802,2.818643
5,muchinga,3.992354,muchinga,2.621986,muchinga,0.197354,0.542903,0.193398,0.507287,3.485066
6,northern,3.990773,northern,2.440774,northern,0.565251,1.088466,0.226154,1.064713,2.926060
7,northwestern,3.987134,northwestern,2.216512,northwestern,-0.269666,1.308641,0.334048,1.265288,2.721846
8,southern,3.982801,southern,1.314307,southern,0.411949,1.821815,0.377549,1.782264,2.200537
9,western,3.981835,western,0.736167,western,0.090999,2.658523,0.276286,2.644128,1.337707


In [1129]:
# # Postharvest and rheas common years 
# post_list_years= postharvest['year'].to_list()
# years_set=set(post_list_years)
# post_harvest_years=list(years_set)
# post_harvest_years

# # df1=rheas_tamsat_province.loc[rheas_tamsat_province['year']==2016]
# rheas_tamsat_province2=pd.DataFrame(columns =[rheas_tamsat_province.columns])
# for i in post_harvest_years:
#     df=rheas_tamsat_province.loc[rheas_tamsat_province['year']==i].reset_index(drop=True)
#     rheas_tamsat_province2=pd.concat([df])


In [1130]:
# Postharvest and rheas metrics
# only data from 2011 to 2014

rheas_tamsat_post_province=rheas_tamsat_province.loc[(rheas_tamsat_province['year']>=2011)|(rheas_tamsat_province['year']<=2014)]
rheas_post_prov_check=[]
province_metrics2=[]
for i in provinces:
    post_province1=post_province.loc[post_province['province_b']==i].reset_index(drop=True)
    rheas_tamsat_province2=rheas_tamsat_province.loc[rheas_tamsat_province['province_b']==i].reset_index(drop=True)

    #  pearson's correlation
    corr=rheas_tamsat_province2['yield'].corr(post_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheas_tamsat_province2['yield']-post_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheas_tamsat_province2['yield']-rheas_tamsat_province2['yield'].mean())
                       -(post_province1['yield']-post_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheas_tamsat_province2['yield']-post_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics2.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])
    rheas_preharvest_prov_check.append(preharvest_province1)

check_rheas_preharvest_prov= pd.concat(rheas_preharvest_prov_check, ignore_index=True)

province_statistics2=pd.DataFrame(province_metrics2)
province_statistics2=province_statistics2.rename({0:'province',1: 
                                                  'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
province_statistics2.to_csv('data/postharvest_rheas_province_level_metrics.csv')
province_statistics2
province_statistics2

KeyError: 'yield'

In [ ]:
# bias correcting tamsat using preharvest over the province
rheas_tamsat_post_province_grouped=rheas_tamsat_post_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province=post_province.groupby(['province_b'],as_index=False).agg({'yield':'mean'})

# marge preharvest, metrics and tamsat rheas yields
province_rheas_preharvest=pd.concat([rheas_tamsat_post_province_grouped,
                                     rheas_tamsat_post_province_grouped,province_statistics2], axis=1)

# calculate bias corrected yield
province_rheas_preharvest['corrected_yield']=province_rheas_preharvest['yield']-province_statistics2['bias']
province_rheas_preharvest

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
# bias correction

# rheas_tamsat_province['yield_corrected']
rheas_tamsat_preharvest_province=rheas_tamsat_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province=post_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province.rename({'province_b':'province_p','yield':'postharvest'},axis=1, inplace=True)
rheas_tamsat_preharvest_province['yield_corrected']=rheas_tamsat_preharvest_province['yield']-province_statistics2['bias']
rheas_postharvest_provincial=pd.concat([rheas_tamsat_preharvest_province,post_province,province_statistics2], axis=1)

rheas_postharvest_provincial1=rheas_postharvest_provincial[['province','yield','postharvest', 'yield_corrected','correlation', 
                              'rmse', 'ubrmse', 'bias']]
rheas_postharvest_provincial1.rename({'yield':'rheas'},axis=1, inplace=True)
rheas_postharvest_provincial1

C:\Users\kasit\AppData\Local\Temp\ipykernel_13436\3299489503.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rheas_postharvest_provincial1.rename({'yield':'rheas'},axis=1, inplace=True)


,province,rheas,postharvest,yield_corrected,correlation,rmse,ubrmse,bias
0,central,4.033719,2.078341,1.892071,0.245991,2.154293,0.298362,2.141648
1,copperbelt,4.038422,2.210160,2.055434,-0.175194,1.991247,0.238252,1.982988
2,eastern,4.035821,1.768208,1.589136,0.636537,2.447042,0.183879,2.446685
3,luapula,4.039153,2.554374,2.367916,-0.025227,1.672983,0.201514,1.671236
4,lusaka,4.033893,2.115439,1.926339,-0.188453,2.110603,0.220497,2.107554
5,muchinga,4.040451,2.621986,2.436592,-0.910857,1.616914,0.276440,1.603859
6,northern,4.038466,2.440774,2.260939,-0.677199,1.794508,0.304953,1.777528
7,northwestern,4.035446,2.216512,2.034894,-0.227523,2.023189,0.352234,2.000553
8,southern,4.033572,1.314307,1.128970,0.613067,2.907625,0.227865,2.904602
9,western,4.036986,0.736167,0.540632,0.460222,3.501665,0.274591,3.496354
